In [1]:
# Train model
import numpy as np
from skimage.color import rgb2gray
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from skimage import filters

def trainModel(X, y):
    size = len(X)
    my_X_gray = rgb2gray(X)

    sobel = [0]*size
    for i in range(size):
        sobel[i] = filters.sobel(my_X_gray[i])
    sobel = np.array(sobel)
    
    my_hog_gradients = [0]*len(my_X_gray)
    my_hog_visuals = [0]*len(my_X_gray)
    
    for i in range(size):
        my_hog_gradients[i], my_hog_visuals[i] = hog(sobel[i], orientations=8, pixels_per_cell=(10, 10), cells_per_block=(4,4),visualize=True)

    my_hog_visuals = np.array(my_hog_visuals)
    my_hog_visuals = my_hog_visuals.reshape(len(my_X_gray), 10000)
    my_hog_gradients = np.array(my_hog_gradients)
    
    pipe_SVC = make_pipeline(TruncatedSVD(n_components=100, random_state=42), SVC(C=20, gamma='scale', kernel='rbf', class_weight='balanced', break_ties=True, probability=True))

    my_X_trainhog, my_X_testhog, my_y_trainhog, my_y_testhog = train_test_split(my_hog_gradients, y, test_size=0.2)

    my_model_SVC_rotate = pipe_SVC.fit(my_X_trainhog, my_y_trainhog.flatten())
    my_y_predtrain = my_model_SVC_rotate.predict(my_X_trainhog)
    my_y_predtest = my_model_SVC_rotate.predict(my_X_testhog)

    my_accuracy_train = accuracy_score(my_y_trainhog, my_y_predtrain)
    my_accuracy_test = accuracy_score(my_y_testhog, my_y_predtest)
    print("Accuracy in the Training set with", my_accuracy_train*100, '%')
    print("Accuracy in the Test set with", my_accuracy_test*100, '%')
    
    return my_model_SVC_rotate

In [2]:
# Test easy set
import numpy as np
from skimage.color import rgb2gray
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import joblib
from skimage import filters
from sklearn.decomposition import TruncatedSVD

def testModelEasy(X, y):    
    model = svm_rotateSobelHOG = joblib.load('svm_rotateSobelHOG.pkl')
    
    size = len(X)
    X_gray = rgb2gray(X)
    
    sobel = [0]*size
    for i in range(size):
        sobel[i] = filters.sobel(X_gray[i])
    sobel = np.array(sobel)
    
    hog_gradients = [0]*size
    hog_visuals = [0]*size
    
    for i in range(size):
        hog_gradients[i], hog_visuals[i] = hog(sobel[i], orientations=8, pixels_per_cell=(10, 10), cells_per_block=(4,4),visualize=True)

    hog_visuals = np.array(hog_visuals)
    hog_visuals = hog_visuals.reshape(size, 10000)
    hog_gradients = np.array(hog_gradients)
    sobel = sobel.flatten().reshape(size, 10000)

    y_predtest = model.predict(hog_gradients)
    y_decision = model.decision_function(hog_gradients)
    
    output = [];
    for i in range (size):
        number = ord(y_predtest[i]) - 64
        output.append(number)
        
    accuracy_test = accuracy_score(y, y_predtest)
    print("Accuracy in the Test set is", accuracy_test*100, '%')
    return output

In [3]:
# Test hard set
import numpy as np
from skimage.color import rgb2gray
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import joblib
from skimage import filters
from sklearn.decomposition import TruncatedSVD

def testModelHard(X, y):    
    model = svm_rotateSobelHOG = joblib.load('svm_rotateSobelHOG.pkl')

    size = len(X)
    X_gray = rgb2gray(X)
    
    sobel = [0]*size
    for i in range(size):
        sobel[i] = filters.sobel(X_gray[i])
    sobel = np.array(sobel)
    
    hog_gradients = [0]*size
    hog_visuals = [0]*size
    
    for i in range(size):
        hog_gradients[i], hog_visuals[i] = hog(sobel[i], orientations=8, pixels_per_cell=(10, 10), cells_per_block=(4,4),visualize=True)

    hog_visuals = np.array(hog_visuals)
    hog_visuals = hog_visuals.reshape(size, 10000)
    hog_gradients = np.array(hog_gradients)
    sobel = sobel.flatten().reshape(size, 10000)

    y_predtest = model.predict(hog_gradients)
    y_decision = model.decision_function(hog_gradients)
    y_proba = model.predict_proba(hog_gradients)
    np.set_printoptions(suppress=True)
    y_proba = np.round(y_proba,4)*100
    
    output = []
    for i in range(size):
        if(y_proba[i][np.argmax(y_proba[i])] < 40):
            output.append(-1)
        else:
            number = ord(y_predtest[i]) - 64
            output.append(number)
    accuracy_test = accuracy_score(y, y_predtest)
    print("Accuracy in the Test set is", accuracy_test*100, '%')
    return output

In [4]:
# Example code
X = np.load('train_data.npy')
y = np.load('train_labels.npy')
print(testModelEasy(X, y))
print('\n')
print(testModelHard(X, y))

Accuracy in the Test set is 100.0 %
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 